<a href="https://colab.research.google.com/github/Jorge2018/prediccion_ventas/blob/main/prediccion_ventas07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
df=pd.read_csv(r'https://raw.githubusercontent.com/Jorge2018/RepositorioArchivos/main/sales_prediction.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


**Inspeccion de datos**

In [230]:
df.duplicated()
df = df.drop_duplicates()
print('la cantidad de registros sin duplicados es:\n',df.count())
print('---------------------------')
print('los datos faltantes del df son:\n',df.isna().sum())
print('---------------------------')

la cantidad de registros sin duplicados es:
 Item_Identifier              8523
Item_Weight                  7060
Item_Fat_Content             8523
Item_Visibility              8523
Item_Type                    8523
Item_MRP                     8523
Outlet_Identifier            8523
Outlet_Establishment_Year    8523
Outlet_Size                  6113
Outlet_Location_Type         8523
Outlet_Type                  8523
Item_Outlet_Sales            8523
dtype: int64
---------------------------
los datos faltantes del df son:
 Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64
---------------------------


In [231]:
df.describe

<bound method NDFrame.describe of      Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0              FDA15        9.300          Low Fat         0.016047   
1              DRC01        5.920          Regular         0.019278   
2              FDN15       17.500          Low Fat         0.016760   
3              FDX07       19.200          Regular         0.000000   
4              NCD19        8.930          Low Fat         0.000000   
...              ...          ...              ...              ...   
8518           FDF22        6.865          Low Fat         0.056783   
8519           FDS36        8.380          Regular         0.046982   
8520           NCJ29       10.600          Low Fat         0.035186   
8521           FDN46        7.210          Regular         0.145221   
8522           DRG01       14.800          Low Fat         0.044878   

                  Item_Type  Item_MRP Outlet_Identifier  \
0                     Dairy  249.8092            OUT04

**Separa valores en X, y**

In [350]:
# Dividan
X = df.drop('Item_Outlet_Sales', axis=1)
#X_sel = X.select_dtypes(include = ["number"])
y = df['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34, random_state=42)

**Instanciar selectores**

In [351]:
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

**Instanciar transformadores SimpleImputer, StandardScaler y OneHotEncoder**

In [352]:
# Imputers
mean_imputer = SimpleImputer(strategy='mean')
freq_imputer = SimpleImputer(strategy='most_frequent')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

**Instanciar el pipeline**

In [353]:
numeric_pipe = make_pipeline(mean_imputer, scaler)
categorical_pipe = make_pipeline(freq_imputer, ohe)

**Asignacion de tupla e instanciacion de ColumnTransformer**

In [354]:
number_tuple = (numeric_pipe, num_selector)
category_tuple = (categorical_pipe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)


**Transformacion de datos**

In [355]:
# fit on train
preprocessor.fit(X_train,X_test)
# transform train and test
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)


**Inspeccion de resultados**

In [356]:
# Comprueben los valores faltantes y que los datos se escalen y tengan una codificación one-hot
print(np.isnan(X_train_processed).sum().sum(), 'valores perdidos en la data de entrenamiento')
print(np.isnan(X_test_processed).sum().sum(), 'valores perdidos en la data de prueba')
print('\n')
print('Todos los datos procesados del X_train_processed son del tipo:', X_train_processed.dtype)
print('Todos los datos procesados del X_test_processed son del tipo', X_test_processed.dtype)
print('\n')
print('shape de los datos de entrenamiento son', X_train_processed.shape)
print('\n')
print('shape de los datos de test son', X_test_processed.shape)


0 valores perdidos en la data de entrenamiento
0 valores perdidos en la data de prueba


Todos los datos procesados del X_train_processed son del tipo: float64
Todos los datos procesados del X_test_processed son del tipo float64


shape de los datos de entrenamiento son (5625, 1584)


shape de los datos de test son (2898, 1584)


**Instancia y proceso del modelo**

In [357]:
reg = LinearRegression()
reg.fit(X_train_processed,y_train)
print('El bias calculado es', reg.intercept_)
print('La pendiente calculada es: ', reg.coef_)

El bias calculado es 2.4148014313624428e+16
La pendiente calculada es:  [-2.85029018e+00 -3.32208999e+01 -5.92832826e+02 ... -3.22957310e+15
  5.34620091e+15 -1.51335733e+16]


Codigo para obtener el R cuadrado (R^2), despues de realizado el ajuste sobre los datos de entrenamiento

In [358]:
train_score = reg.score(X_train_processed, y_train)
print(train_score)

0.6790689435788477


Codigo para obtener el R cuadrado (R^2), despues de realizado el ajuste sobre los datos de prueba

In [359]:
test_score = reg.score(X_test_processed,y_test) # Este valor sale negativo no esta claro el porque
print(test_score)

-9.72722541423288e+20


In [349]:
# Obtengan las predicciones del conjunto de entrenamiento
train_preds = reg.predict(X_train_processed)
# Obtengan las predicciones del conjunto de prueba
test_preds = reg.predict(X_test_processed)

Error medio absoluto

In [329]:
mae_train = mean_absolute_error(y_train, X_train_processed) # Como se interpretan estos valores?
mae_test = mean_absolute_error(y_test, X_test_processed)    # Como se interpretan estos valores?
print(mae_train)
print(mae_test)

ValueError: ignored

Error cuadratico medio

In [269]:
mse_train = mean_squared_error(y_train, train_preds) # Como se interpretan estos valores?
mse_test = mean_squared_error(y_test, test_preds)    # Como se interpretan estos valores?
print(mse_train)
print(mse_test)

ValueError: ignored

In [270]:
rmse_train = np.sqrt(mean_squared_error(y_train, train_preds)) # Como se interpretan estos valores?
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))    # Como se interpretan estos valores?
print(rmse_train)
print(rmse_test)

ValueError: ignored